<a href="https://colab.research.google.com/github/Gressling/notebooks/blob/main/Mixed_Integer_Optimization_for_Cost_Effective_Chemistry_Production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# title: (MIP) Optimization: Compare two Production Scenarios
# author: Gressling, T                               # license: MIT License
# github: github.com/gressling/notebooks               # activity: single example
# indices: [wiki.Q6042592]

# description: Solve a mixed-integer programming problem for a chemistry production scenario.
# It minimizes production costs while meeting demand and resource constraints,
# ensuring both processes are used by incorporating fixed costs, binary variables, and minimum production constraints.


In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 34.7 MB/s eta 0:00:00


In [11]:
import pulp

# Define the problem
prob = pulp.LpProblem("Chemistry_Production", pulp.LpMinimize)

# Decision variables
# x1 and x2 are the amounts of chemicals A and B produced in process 1 (continuous variables)
x1 = pulp.LpVariable('x1', lowBound=0, cat='Continuous')
x2 = pulp.LpVariable('x2', lowBound=0, cat='Continuous')

# y1 and y2 are the amounts of chemicals A and B produced in process 2 (integer variables)
y1 = pulp.LpVariable('y1', lowBound=0, cat='Integer')
y2 = pulp.LpVariable('y2', lowBound=0, cat='Integer')

# Binary variables to indicate if a process is used
z1 = pulp.LpVariable('z1', cat='Binary')
z2 = pulp.LpVariable('z2', cat='Binary')

# Define the cost coefficients for the processes
cost_process_1 = [5, 4]  # Variable costs to produce A and B in process 1
cost_process_2 = [3, 2]  # Reduced variable costs to produce A and B in process 2

# Fixed costs to use each process
fixed_cost_process_1 = 50
fixed_cost_process_2 = 80

# Objective function: Minimize total production cost including fixed costs
prob += (cost_process_1[0] * x1 + cost_process_1[1] * x2 +
         cost_process_2[0] * y1 + cost_process_2[1] * y2 +
         fixed_cost_process_1 * z1 + fixed_cost_process_2 * z2), "Total_Cost"

# Constraints
# Production capacity constraints
prob += x1 + y1 >= 50, "Demand_A"  # Demand for chemical A
prob += x2 + y2 >= 40, "Demand_B"  # Demand for chemical B

# Resource constraints
prob += 2 * x1 + 3 * x2 + 2 * y1 + 3 * y2 <= 300, "Resource_Availability"

# Minimum usage constraints for both processes
prob += z1 == 1, "Process_1_Used"
prob += z2 == 1, "Process_2_Used"

# Ensure binary variables are correctly linked to production variables
prob += x1 <= 1000 * z1, "Link_x1_z1"
prob += x2 <= 1000 * z1, "Link_x2_z1"
prob += y1 <= 1000 * z2, "Link_y1_z2"
prob += y2 <= 1000 * z2, "Link_y2_z2"

# New minimum production constraints for both processes
prob += x1 >= 10 * z1, "Min_Production_Process_1_A"
prob += x2 >= 10 * z1, "Min_Production_Process_1_B"
prob += y1 >= 10 * z2, "Min_Production_Process_2_A"
prob += y2 >= 10 * z2, "Min_Production_Process_2_B"

# Solve the problem
prob.solve()

# Print the results
print("Status:", pulp.LpStatus[prob.status])

print("Optimal production of chemical A in process 1:", pulp.value(x1))
print("Optimal production of chemical B in process 1:", pulp.value(x2))
print("Optimal production of chemical A in process 2:", pulp.value(y1))
print("Optimal production of chemical B in process 2:", pulp.value(y2))
print("Process 1 used:", pulp.value(z1))
print("Process 2 used:", pulp.value(z2))
print("Total cost:", pulp.value(prob.objective))


Status: Optimal
Optimal production of chemical A in process 1: 10.0
Optimal production of chemical B in process 1: 10.0
Optimal production of chemical A in process 2: 40.0
Optimal production of chemical B in process 2: 30.0
Process 1 used: 1.0
Process 2 used: 1.0
Total cost: 400.0
